In [ ]:
#CSV to GEOJSON
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Load csv
file_path = '../model_domain/irma_observation_locations.csv'
df = pd.read_csv(file_path)
# create geometry
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
# create geo dataframe
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
# set name as index for id
gdf.set_index('obs #', inplace=True)

# Export
output_path = "../model_domain/obs_points_savannah_irma.geojson"
gdf.to_file(output_path, driver='GeoJSON')

In [15]:
#GPKG TO GEOJSON
import geopandas as gpd
import os

def convert_gpkg_to_geojson(gpkg_path, geojson_path, layer_name=None):
    """
    Zet een GeoPackage (.gpkg) om naar een GeoJSON bestand.
    
    Parameters:
    - gpkg_path: Pad naar het input .gpkg bestand.
    - geojson_path: Pad waar de .geojson opgeslagen moet worden.
    - layer_name: (Optioneel) De naam van de specifieke laag in de gpkg.
    """
    
    # Controleer of het inputbestand bestaat
    if not os.path.exists(gpkg_path):
        print(f"Error: File '{gpkg_path}' not found.")
        return

    try:
        print("Reading file...")
        
        # Lees de gpkg. Als er een laagnaam is, gebruik die.
        # Anders pakt GeoPandas automatisch de eerste laag.
        if layer_name:
            gdf = gpd.read_file(gpkg_path, layer=layer_name)
        else:
            gdf = gpd.read_file(gpkg_path)

        print(f"Data read: {len(gdf)} rows.")

        # Schrijf naar GeoJSON
        # driver='GeoJSON' forceert het juiste formaat
        gdf.to_file(geojson_path, driver='GeoJSON')
        
        print(f"Success! File saved as: {geojson_path}")

    except Exception as e:
        print(f"Something went wrong: {e}")

# --- Voorbeeld gebruik ---
input_bestand = "../data_processed/seawalk.gpkg"
output_bestand = "../data_processed/seawalk_levee.geojson"

# Voer de conversie uit
convert_gpkg_to_geojson(input_bestand, output_bestand)

Reading file...
Data read: 1 rows.
Success! File saved as: ../data_processed/seawalk_levee.geojson


In [ ]:
#MERGE SIMULATED TIMESERIES
import pandas as pd
from IPython.display import display

paths = ["../data/water_levels/design_hydrographs_full_8670870.csv",
         "../data/water_levels/design_hydrographs_full_8720218.csv"]
T = 500

data = []
for path in paths:
    id = path.rsplit("_", 1)[-1].replace(".csv", "")
    df = pd.read_csv(path)

    col = str(T)
    df_T = df[['time', col]]
    df_T.columns = ["time", id]

    data.append(df_T)

merged_df = (pd.concat([df.set_index("time") for df in data], axis=1))
display(merged_df)

# merged_df.to_csv(f"../data/water_levels/sim_timeseries_T{T}.csv")




,8670870,8720218
time,,
2021-01-01 00:00:00,0.76,0.76
2021-01-01 01:00:00,0.66,0.66
2021-01-01 02:00:00,0.40,0.40
2021-01-01 03:00:00,0.04,0.04
2021-01-01 04:00:00,-0.33,-0.33
...,...,...
2021-01-14 20:00:00,0.17,0.07
2021-01-14 21:00:00,0.54,0.44
2021-01-14 22:00:00,0.84,0.74


2.33
1.72
